In [1]:
import pandas as pd
import matplotlib.pyplot as plt
# from profiler import profiler
from mlxtend.frequent_patterns import apriori, association_rules
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

# CREATE DATAFRAME
df = pd.read_excel("Megastore Dataset.xlsx")

# NO DUPLICATE ROWS FOUND
# df = df.drop_duplicates()

c:\Users\ajmar\miniconda3\Lib\site-packages\openpyxl\packaging\core.py:99: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.datetime.utcnow()
c:\Users\ajmar\miniconda3\Lib\site-packages\openpyxl\packaging\core.py:99: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.datetime.utcnow()


In [2]:
# DEFINE ENCODED VARIABLES 
ordinal_variables = ['Order Priority', 'Segment']
nominal_variables = ['Category', 'Region']

# ORDINAL ENCODING
ordinal_encoder = OrdinalEncoder(categories=[['Low','Medium','High','Critical'],['Consumer','Home Office','Corporate']])
df[ordinal_variables] = ordinal_encoder.fit_transform(df[ordinal_variables])

# ONE-HOT ENCODING
df = pd.get_dummies(df, columns=nominal_variables)

In [3]:
# TRANSACTIONALIZE INTO BINARY MATRIX
basket = (df.groupby(['Order ID', 'Product Name'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('Order ID'))


In [4]:
basket = basket.map(lambda x: 1 if x > 0 else 0)

In [5]:
basket.to_csv("prepared_megastore_dataset.csv")

In [6]:
# Apriori algorithm & generate association rules

frequent_itemsets = apriori(basket, min_support=0.001, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.2)

# Export association rules to CSV file
rules.to_csv("association_rules.csv")

# Print a success message to ensure the code runs without errors
print("Apriori algorithm executed successfully, and association rules generated.")

c:\Users\ajmar\miniconda3\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


Apriori algorithm executed successfully, and association rules generated.


In [7]:
print(rules.head())
print(rules.shape)
print(frequent_itemsets.head())
print(frequent_itemsets.shape)
print(basket.info())
print(basket.describe())


Empty DataFrame
Columns: [antecedents, consequents, antecedent support, consequent support, support, confidence, lift, leverage, conviction, zhangs_metric]
Index: []
(0, 10)
    support                       itemsets
0  0.001438     (Acco 3-Hole Punch, Clear)
1  0.001594   (Acco 3-Hole Punch, Durable)
2  0.001710   (Acco 3-Hole Punch, Economy)
3  0.001205  (Acco 3-Hole Punch, Recycled)
4  0.001632    (Acco Binder Covers, Clear)
(507, 2)
<class 'pandas.core.frame.DataFrame'>
Index: 25728 entries, AE-2012-PO8865138-41184 to ZA-2015-TS11205146-42062
Columns: 3788 entries, "While you Were Out" Message Book, One Form per Page to netTALK DUO VoIP Telephone Service
dtypes: int64(3788)
memory usage: 743.7+ MB
None
Product Name  "While you Were Out" Message Book, One Form per Page  \
count                                              25728.000000      
mean                                                   0.000117      
std                                                    0.010798      
min 